In [1]:
# 引用套件
import boto3

In [2]:
# 宣告變數
table_Name = 'vcloudlab_table'    # dynamodb table 名稱

In [3]:
# 使用 client & resource 方法連線dynamodb
dynamodb = boto3.resource('dynamodb',
                          endpoint_url='http://dynamodb.vcloudlab.pro:8000',                        
                         verify=False               #驗證ssl證書功能，預設SSL證書需要驗證，設定verify=False不去驗證SSL證書的有效性 
                         )

In [4]:
# 創建table
create_table = dynamodb.create_table(
    TableName= table_Name, #命名為vcloudlab_table
    KeySchema=[
        {
            'AttributeName': 'username',
            'KeyType': 'HASH'     #主key
        },
        {
            'AttributeName': 'last_name',
            'KeyType': 'RANGE'    #sortkey
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'username',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'last_name',
            'AttributeType': 'S'
        },
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 50,
        'WriteCapacityUnits': 50
    }
)


In [5]:
# 計算 table的物件數
res = dynamodb.Table(table_Name).scan(AttributesToGet=['username'])
print(res['Count'])

0


In [6]:
# 對已存在的table建立連線
table = dynamodb.Table(table_Name)
# 顯示插入資料之前的item數量
print('beforeput:', table.item_count)
# 插入資料
table.put_item(
    Item={'username': 'apple2',
          'last_name': 'banana',
          'age': 30,
          'account_type': 'standard_user'
         }
            )
# 顯示插入資料之後的item數量
print('afterput:', table.item_count)

beforeput: 0
afterput: 1


In [7]:
#查詢資料
response = table.get_item(
Key={
'username': 'apple2',
'last_name': 'banana'
}
)
item = response['Item']
print(item)


{'last_name': 'banana', 'account_type': 'standard_user', 'age': Decimal('30'), 'username': 'apple2'}


In [8]:
#更新物件 
table.update_item(
Key={
'username': 'apple',
'last_name': 'banana'
},
UpdateExpression='SET age = :val1', #更新表達式
ExpressionAttributeValues={ #表達式屬性值
':val1': 60
}
)


{'ResponseMetadata': {'RequestId': 'd3a1f76a-d6cc-459b-9fc0-588121919e8b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '2745614147',
   'x-amzn-requestid': 'd3a1f76a-d6cc-459b-9fc0-588121919e8b',
   'content-length': '2',
   'server': 'Jetty(8.1.12.v20130726)'},
  'RetryAttempts': 0}}

In [9]:
# 更新後查詢資料 
response = table.get_item(
        Key={
            'username': 'apple',
             'last_name': 'banana'
            }
)
item = response['Item']
print(item)


{'last_name': 'banana', 'age': Decimal('60'), 'username': 'apple'}


In [10]:
# 刪除資料
print('beforedelete:', table.item_count)
table.delete_item(
    Key={
        'username': 'apple',
        'last_name': 'banana'
    }
)
print('afterdelete:', table.item_count)


beforedelete: 2
afterdelete: 1


In [11]:
# 刪除資料表
table = dynamodb.Table('vcloudlab_table')
table.delete()

{'TableDescription': {'AttributeDefinitions': [{'AttributeName': 'username',
    'AttributeType': 'S'},
   {'AttributeName': 'last_name', 'AttributeType': 'S'}],
  'TableName': 'vcloudlab_table',
  'KeySchema': [{'AttributeName': 'username', 'KeyType': 'HASH'},
   {'AttributeName': 'last_name', 'KeyType': 'RANGE'}],
  'TableStatus': 'ACTIVE',
  'CreationDateTime': datetime.datetime(2019, 7, 31, 5, 16, 8, 424000, tzinfo=tzlocal()),
  'ProvisionedThroughput': {'LastIncreaseDateTime': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzlocal()),
   'LastDecreaseDateTime': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzlocal()),
   'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 50,
   'WriteCapacityUnits': 50},
  'TableSizeBytes': 59,
  'ItemCount': 1,
  'TableArn': 'arn:aws:dynamodb:ddblocal:000000000000:table/vcloudlab_table',
  'BillingModeSummary': {'BillingMode': 'PROVISIONED',
   'LastUpdateToPayPerRequestDateTime': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzlocal())}},
 'ResponseMeta